In [1]:
import pandas as pd
import os

In [2]:
#Get all the files from the os
file_list = os.listdir('./data')
print(f"Data for years: {[file_name[:4] for file_name in file_list]} collected")

Data for years: ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025'] collected


In [3]:
#Parse folder for csv files and combine
df_list = []

for file_name in file_list:
    path = os.path.join('data', file_name)
    df = pd.read_csv(path, dtype={'url': str})

    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)
print(f"Combined into one dataset of shape {combined_df.shape}")
print(f"Combined dataset info: ")
print(combined_df.info())
print(combined_df['dragons'].head())


Combined into one dataset of shape (1077780, 164)
Combined dataset info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077780 entries, 0 to 1077779
Columns: 164 entries, gameid to opp_deathsat25
dtypes: float64(131), int64(10), object(23)
memory usage: 1.3+ GB
None
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: dragons, dtype: float64


Check incompleteness

In [4]:
print(len(combined_df['elementaldrakes'].isnull()))
print(len(combined_df['gameid']))
#infernals,mountains,clouds,oceans,chemtechs,hextechs
print(len(combined_df['infernals'].isnull()))
print(len(combined_df['mountains'].isnull()))
print(len(combined_df['clouds'].isnull()))
print(len(combined_df['oceans'].isnull()))
print(len(combined_df['chemtechs'].isnull()))
print(len(combined_df['hextechs'].isnull()))
print(len(combined_df['opp_dragons'].isnull()))
print(len(combined_df['elders'].isnull()))
print(len(combined_df['opp_elders'].isnull()))

1077780
1077780
1077780
1077780
1077780
1077780
1077780
1077780
1077780
1077780
1077780


In [5]:

i = 0
for val in combined_df['datacompleteness']:
    if val == 'partial':
        i += 1
print(f"{100*(i/combined_df.shape[0])}% of rows are partially complete")
print(i)
incomplete_indices = df.index[df['datacompleteness'] == 'partial'].tolist()




11.7797695262484% of rows are partially complete
126960


In [6]:
#create player and team map 
player_map = combined_df.set_index('playerid')['playername'].to_dict()

team_map = combined_df.set_index('teamid')['teamname'].to_dict()

In [7]:
#Delete the url column since it is unneeded 
print(combined_df['url'].tail(20))
combined_df = combined_df.drop('url', axis=1)
print(combined_df.shape)


1077760    NaN
1077761    NaN
1077762    NaN
1077763    NaN
1077764    NaN
1077765    NaN
1077766    NaN
1077767    NaN
1077768    NaN
1077769    NaN
1077770    NaN
1077771    NaN
1077772    NaN
1077773    NaN
1077774    NaN
1077775    NaN
1077776    NaN
1077777    NaN
1077778    NaN
1077779    NaN
Name: url, dtype: object
(1077780, 163)


In [8]:
assert team_map['oe:team:47ae4f5f4aea5a7a0ab0b9778844cc2'] == 'Fnatic Academy'

In [9]:
unique_game_id = combined_df['gameid'].unique()


filtered_df = combined_df[combined_df['datacompleteness'] == 'complete']
filtered_unique_games = filtered_df['gameid'].unique()

filtered_players = filtered_df['playerid'].unique()
filtered_teams = filtered_df['teamid'].unique()

print(f"We have {len(unique_game_id)} unique games of which {len(filtered_unique_games)} have complete data")
print(f"Looking at complete data we have {len(filtered_teams)} unique teams consisting of {len(filtered_players)} unique players")

We have 89815 unique games of which 79235 have complete data
Looking at complete data we have 1866 unique teams consisting of 9639 unique players


Now we want to transform it into dataframes: game_metadata, game_player_stats
somehow for each match and match id there is one player without name and id assuming it is their coach  ?
gameid is the unique id of each game while game is only the number of game in each series however there is not really a way to get which series the game is from (just drop the game?)

Game_metadata : Consist of all unqiue games with their all relevant information (primary key gameid)
game_player_stats: Matches a player and game together and holds all information about the stats of the given player in a given game (primary key : (gameid, playerid)) 

Note: Only includes games with datacompletness == complete thus using the filteredf

In [10]:
#game_metadata
game_metadata = filtered_df[['gameid', 'date', 'league', 'playoffs', 'patch', 'gamelength']].drop_duplicates(subset='gameid')

#game_player_stats
game_player_stats = filtered_df[[
    'gameid', 'playerid', 'teamid', 'side', 'position', 'champion', 'result', 'kills', 'deaths', 'assists', 
    'damagetochampions', 'visionscore', 'earnedgold', 'total cs', 'golddiffat15', 'csdiffat15', 'xpdiffat15'
    ]]




In [11]:
print(game_player_stats['xpdiffat15'].unique())
print(game_player_stats['xpdiffat15'].dtype)


[ -560.  -703.  -148. ... -5178.  6059. -6059.]
float64


In [12]:
print(game_metadata['gamelength'].unique())
print(game_metadata['gamelength'].dtype)
has_decimal = (game_player_stats['golddiffat15'] % 1 != 0).any()

print("Any values with decimals:", has_decimal)

[1924 2474 2629 ... 3119 1188 3414]
int64
Any values with decimals: True


In [13]:
print(f"Correcting patches, currently {len(game_metadata[game_metadata['patch'].isnull()])} values missing")
# We want to fix the missing patches possibly by infering the patch from games on a similar date
#Try to convert it to datetime first
game_metadata['date'] = pd.to_datetime(game_metadata['date'], errors='coerce')
print(game_metadata['date'].isnull().sum(), "dates could not be converted and are now NaT")

#split into know patches
known_patch = game_metadata[game_metadata['patch'].notnull()]
missing_patch = game_metadata[game_metadata['patch'].isnull()]

assert (len(known_patch) + len(missing_patch) == len(game_metadata))

#Get nearby dates to infer patch
df_known_sorted = known_patch.sort_values('date')
df_missing_sorted = missing_patch.sort_values('date')

tolerance_days = pd.Timedelta(days=20)

#Use merge_asof to merge df_known and df_missing on date as a key looking for the nearest key given there is one within the tolerance
df_merged = pd.merge_asof(df_missing_sorted, df_known_sorted[['date', 'patch']], on='date', tolerance=tolerance_days, direction='nearest')

for _, row in df_merged.iterrows():
    patch_val = game_metadata.loc[game_metadata['gameid'] == row['gameid'], 'patch'].iloc[0]
    if pd.isna(patch_val):
        correct_patch = row['patch_y']
        game_metadata.loc[game_metadata['gameid'] == row['gameid'], 'patch'] = row['patch_y']       

print(f"After correction {len(game_metadata[game_metadata['patch'].isnull()])} values missing")
print(f"That is: {63 / len(game_metadata)}%")

print("Deleting rows with patch still nan")
missing_patch = game_metadata[game_metadata['patch'].isnull()]
#assert len(missing_patch) == 63

game_metadata = game_metadata.dropna(subset=['patch'])

#This basically says if the gameid of a row is not a gameid in the missing_patch keep the row and delete it otherwise since ~ works as negation
game_player_stats = game_player_stats[~game_player_stats['gameid'].isin(missing_patch['gameid'])]

Correcting patches, currently 179 values missing
0 dates could not be converted and are now NaT
After correction 63 values missing
That is: 0.0007951031741023538%
Deleting rows with patch still nan


In [14]:
print(game_metadata['date'].unique())

<DatetimeArray>
['2014-01-14 17:52:02', '2014-01-14 19:16:29', '2014-01-14 21:28:45',
 '2014-01-14 22:37:18', '2014-01-14 23:48:12', '2014-01-15 17:53:47',
 '2014-01-15 19:15:40', '2014-01-15 21:13:58', '2014-01-15 22:36:24',
 '2014-01-16 18:33:27',
 ...
 '2025-07-24 08:05:02', '2025-07-24 08:56:45', '2025-07-24 10:11:20',
 '2025-07-24 11:15:19', '2025-07-24 14:27:30', '2025-07-24 15:30:58',
 '2025-07-24 16:33:35', '2025-07-24 17:03:51', '2025-07-24 17:42:29',
 '2025-07-24 18:01:00']
Length: 78996, dtype: datetime64[ns]


In [15]:
#Filter the playerids to remove or restore null values 
#First we remove any entry where the position is 'team' assuming its coaches
game_player_stats = game_player_stats[game_player_stats['position'] != 'team']

#print("We have 9313 remaining unknown players")
#print(f"{100*(9313 / len(game_metadata['gameid']))}")
#We could either delete 10% of our games or handle it differently somehow 
#Create a dummy player that somehow participated in 9000 something games 

game_player_stats['playerid'] = game_player_stats['playerid'].fillna('unknown')
player_map['unknown'] = 'this player is unknown'

Either we delete the games with teamID is null or we get through some other database the missing teamIDs because these are random matches where the match at that date somehow disappeared or there are teams that dont exist on the site we have the games from 
wer would lose about 2533 but since we have still over 70 thousand after deleting those that shouldnt be a probglem

In [16]:
#missing team ids
#print(game_player_stats.isnull().sum())

#There are 12665 entries with a missing team id just deleting them would cost 12665 / 5 = 2533 games
#print(game_player_stats[game_player_stats['gameid'] == 'FRA1TMNT1/250030'].tail())
#oe:player:6d09c5240e2126fea4698be0188ba61  FRA1TMNT1/250030 head
#oe:player:0e94a953ebc2e75ab1d22879fcb8d72  LOLTMNT06_129104 tail

#check out head
#missing team of this matchup
#print(game_metadata[game_metadata['gameid'] == 'FRA1TMNT1/250030'])
#print(player_map['oe:player:6d09c5240e2126fea4698be0188ba61'])
#Missing team is Shantao Macht Wau
#Lets look how many times its missing 
#test = game_player_stats[game_player_stats['teamid'].isnull()]
#print(len(test[test['playerid'] == 'oe:player:6d09c5240e2126fea4698be0188ba61']))
#print(test[test['gameid']=='LOLTMNT03_271689'].head())
#other team 
#print(team_map['oe:team:1d16ed15056eb83a39b0863b72cf2ad'])

#check out tail

#print(game_metadata[game_metadata['gameid'] == 'LOLTMNT03_271689'])
#print(player_map['oe:player:920ce0a623916ab3eb57790f9cd3049'])
#print(game_player_stats[game_player_stats['gameid'] == 'LOLTMNT06_129104'].tail(20))
#print(f"other team: {team_map['oe:team:35d740411d9f9072f939b1faf33a6e2']}")

In [17]:
#print(game_player_stats.isnull().sum())
#for the diff at 15 we could use a -1 to indicate the match didnt go as long but its just 60 entries so 12 games
#damagetochampions also an error value should be fine 
#visionscore same probaly doesnt hurt much since we dont really need it so just add dummy value 
#same for earned gold
#Only problematic is teamid
#Lets say we want to delete the ones with missing team ids 

#We want to get a list of gameids of the games where there is atleast one team id missing at maximum that should result in 2556 or something games
missing_teamid = game_player_stats[game_player_stats['teamid'].isnull()]
missing_teamid = missing_teamid['gameid'].unique()
print(len(missing_teamid))
print(len(game_metadata))
print(f"So after cleaning we should have {len(game_metadata) - len(missing_teamid)} entries remaining in gamemetadata")

# The ~ is a negating operator so this means we keep all entries where the gameid is NOT in the list_gameids same for game_metadata 
game_player_stats_clean = game_player_stats[~game_player_stats['gameid'].isin(missing_teamid)]

game_metadata_clean = game_metadata[~game_metadata['gameid'].isin(missing_teamid)]

print(f"After deleting we have {len(game_metadata_clean)}")

#Check for remaining null values
#print(game_player_stats_clean.isnull().sum())

#Transfer changes to main files
game_player_stats = game_player_stats_clean
game_metadata = game_metadata_clean

2130
79172
So after cleaning we should have 77042 entries remaining in gamemetadata
After deleting we have 77042


In [18]:
missing_stats_15min = game_player_stats[game_player_stats['golddiffat15'].isnull()]
missing_stats_15min = missing_stats_15min['gameid'].unique()
#drop the five games which didnt even go till minute 15
game_metadata = game_metadata[~game_metadata['gameid'].isin(missing_stats_15min)]
game_player_stats = game_player_stats[~game_player_stats['gameid'].isin(missing_stats_15min)]
print(len(game_metadata))
print(game_player_stats.isnull().sum())

77037
gameid                   0
playerid                 0
teamid                   0
side                     0
position                 0
champion                 0
result                   0
kills                    0
deaths                   0
assists                  0
damagetochampions      200
visionscore          28500
earnedgold             820
total cs                 0
golddiffat15             0
csdiffat15               0
xpdiffat15               0
dtype: int64


In [19]:
#Games where damagetochampions is missing 
missing_stats_15min = game_player_stats[game_player_stats['damagetochampions'].isnull()]
missing_stats_15min = missing_stats_15min['gameid'].unique()
print(len(missing_stats_15min))
game_metadata = game_metadata[~game_metadata['gameid'].isin(missing_stats_15min)]
game_player_stats = game_player_stats[~game_player_stats['gameid'].isin(missing_stats_15min)]
#print(game_player_stats.isnull().sum())

20


In [20]:
missing_earned_gold = game_player_stats[game_player_stats['earnedgold'].isnull()]
missing_earned_gold = missing_earned_gold['gameid'].unique()
print(len(missing_earned_gold))

game_metadata = game_metadata[~game_metadata['gameid'].isin(missing_earned_gold)]
game_player_stats = game_player_stats[~game_player_stats['gameid'].isin(missing_earned_gold)]
print(game_player_stats.isnull().sum())

82
gameid                   0
playerid                 0
teamid                   0
side                     0
position                 0
champion                 0
result                   0
kills                    0
deaths                   0
assists                  0
damagetochampions        0
visionscore          28300
earnedgold               0
total cs                 0
golddiffat15             0
csdiffat15               0
xpdiffat15               0
dtype: int64


In [21]:
missing_vision_score = game_player_stats[game_player_stats['visionscore'].isnull()]
missing_vision_score = missing_vision_score['gameid'].unique()
print(len(game_metadata))
game_metadata = game_metadata[~game_metadata['gameid'].isin(missing_vision_score)]
game_player_stats = game_player_stats[~game_player_stats['gameid'].isin(missing_vision_score)]
print(len(game_metadata))


76935
74105


In [22]:
print(game_metadata)

                   gameid                date  league  playoffs  patch  \
0                TRLH3/33 2014-01-14 17:52:02  EU LCS         0   3.15   
12               TRLH3/44 2014-01-14 19:16:29  EU LCS         0   3.15   
24               TRLH3/76 2014-01-14 21:28:45  EU LCS         0   3.15   
36               TRLH3/85 2014-01-14 22:37:18  EU LCS         0   3.15   
48            TRLH3/10072 2014-01-14 23:48:12  EU LCS         0   3.15   
...                   ...                 ...     ...       ...    ...   
1077720  LOLTMNT04_125190 2025-07-24 15:30:58     TCL         0  15.14   
1077732  LOLTMNT04_125215 2025-07-24 16:33:35     TCL         0  15.14   
1077744  LOLTMNT04_125221 2025-07-24 17:03:51     EBL         0  15.14   
1077756  LOLTMNT04_125230 2025-07-24 17:42:29     TCL         0  15.14   
1077768  LOLTMNT04_125238 2025-07-24 18:01:00     EBL         0  15.14   

         gamelength  
0              1924  
12             2474  
24             2629  
36             1837  
4

In [23]:

#combined_df.to_csv('combined.csv', index=False)
#print("Saved resulting csv in ./combined.csv")
print(len(game_metadata['gameid'].unique()))
print(game_metadata.isnull().sum())
print(game_player_stats.isnull().sum())
tesst = game_player_stats[game_player_stats['visionscore'].isnull()]
print(len(tesst['gameid'].unique()))
print("Saving game_player_stats into game_player_stats.csv....")
game_player_stats.to_csv('./out/game_player_stats.csv', index=False)
print("Saved game_player_stats")

print("Saving game_metadata into game_metadata.csv.....")
game_metadata.to_csv('./out/game_metadata.csv', index=False)
print("Saved game_metadata")


74105
gameid        0
date          0
league        0
playoffs      0
patch         0
gamelength    0
dtype: int64
gameid               0
playerid             0
teamid               0
side                 0
position             0
champion             0
result               0
kills                0
deaths               0
assists              0
damagetochampions    0
visionscore          0
earnedgold           0
total cs             0
golddiffat15         0
csdiffat15           0
xpdiffat15           0
dtype: int64
0
Saving game_player_stats into game_player_stats.csv....
Saved game_player_stats
Saving game_metadata into game_metadata.csv.....
Saved game_metadata


In [24]:
game_metadata = pd.read_csv('./out/game_metadata.csv')
game_player_stats = pd.read_csv('./out/game_player_stats.csv')
print(game_player_stats['result'].dtype)

int64


In [2]:
import pickle

#with open('teammap.pkl', 'wb') as f:
    #pickle.dump(team_map, f)
#with open('playermap.pkl', 'wb') as f:
    #pickle.dump(player_map, f)

In [ ]:
with open('teammap.pkl', 'rb') as f:
    team_map = pickle.load(f)

print(team_map['oe:team:581e5fa873dd0139907fe323959e32a'])

Gen.G
